![display relevant image here](path/url/to/image)
- Banner/header image

# Title
- Relevant to Data and Business Context

## Overview
- BLUF (Bottom Line Up Front)
- One paragraph summary of findings and analysis
- Frame your 'story'

## Business Understanding
- Set the stage for analysis
- Why are these findings relevant/important?
- Introduce stakeholders
- Postulate about questions you want to ask/answer

## Data Understanding
- Present the source of data
- Describe the data available
- What is relevant to keep what is not
- Present any data cleaning that needs to happen
- Null values? Type mismatches? Duplicates?

### Project Initialization and Data Acquisition

In [1]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import when, regexp_replace, col, trim, lower, split

In [2]:
spark = SparkSession.builder \
    .appName("Google PlayStore App Analysis") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
df = spark.read.csv("google_play_store_dataset.csv", header=True, inferSchema=True)

# 1st validation
df.show(5) 

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [4]:
# 2nd validation
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



<div style="background-color:#fff5e6; padding:15px; border-radius:8px">
No prompts used in data understanding, everything was taken from practices M3 and M4

### Bussiness Questions / Business Context Development

1) Is there a relationship between app size and rating?
2) Can we identify underperforming categories (low installs, low ratings)?
3) What genres tend to perform better in terms of user engagement (ratings)?

In alignment with the structured approach to question development and analysis planning, I selected three progressively complex questions that focus on uncovering key performance drivers within the Google Play Store dataset:

Is there a relationship between app size and rating?
This question explores a potential correlation between app technical characteristics and user satisfaction. It serves as a foundational analysis to identify whether larger or smaller apps tend to be rated higher by users.

Can we identify underperforming categories (low installs, low ratings)?
This diagnostic question aims to detect categories that may be underdelivering on both popularity and quality, helping stakeholders target improvement or resource reallocation efforts.

What genres tend to perform better in terms of user engagement (ratings)?
By evaluating user engagement across different genres, this question supports strategic decisions around app development and marketing focus, offering insights into content types that resonate most with users.

Each question was selected to balance analytical depth with actionable business insights, and mapped to available data attributes such as Rating, Size, Category, Installs, and Genres. Together, they represent a thoughtful progression from exploratory correlation analysis to strategic performance evaluation.

### Data Discovery and Understanding / EDA 

#### Statistical Analysis

In [5]:
# Descriptive stats
df.describe().show()

+-------+--------------------+-------------+-----------+-----------------+------------------+------------------+-----+------------------+--------------+------+-----------------+-------------+------------------+
|summary|                 App|     Category|     Rating|          Reviews|              Size|          Installs| Type|             Price|Content Rating|Genres|     Last Updated|  Current Ver|       Android Ver|
+-------+--------------------+-------------+-----------+-----------------+------------------+------------------+-----+------------------+--------------+------+-----------------+-------------+------------------+
|  count|               10841|        10841|      10841|            10841|             10841|             10841|10841|             10841|         10840| 10841|            10841|        10840|             10840|
|   mean|                NULL|          1.9|        NaN|444225.1924709356|              NULL| 2.866666666666667|  NaN|               0.0|          NULL|  NU

In [6]:
df.groupBy("Category").count().orderBy("count", ascending=False).show()

+-------------------+-----+
|           Category|count|
+-------------------+-----+
|             FAMILY| 1972|
|               GAME| 1144|
|              TOOLS|  843|
|            MEDICAL|  463|
|           BUSINESS|  460|
|       PRODUCTIVITY|  424|
|    PERSONALIZATION|  392|
|      COMMUNICATION|  387|
|             SPORTS|  384|
|          LIFESTYLE|  382|
|            FINANCE|  366|
| HEALTH_AND_FITNESS|  340|
|        PHOTOGRAPHY|  335|
|             SOCIAL|  295|
| NEWS_AND_MAGAZINES|  283|
|           SHOPPING|  260|
|   TRAVEL_AND_LOCAL|  258|
|             DATING|  234|
|BOOKS_AND_REFERENCE|  231|
|      VIDEO_PLAYERS|  175|
+-------------------+-----+
only showing top 20 rows



In [7]:
df.groupBy("Content Rating").count().orderBy("count", ascending=False).show()

+---------------+-----+
| Content Rating|count|
+---------------+-----+
|       Everyone| 8713|
|           Teen| 1208|
|     Mature 17+|  498|
|   Everyone 10+|  414|
|Adults only 18+|    3|
|        Unrated|    2|
|     5,000,000+|    1|
|           NULL|    1|
|     1,000,000+|    1|
+---------------+-----+



In [8]:
df.groupBy("Genres").count().orderBy("count", ascending=False).show()

+-----------------+-----+
|           Genres|count|
+-----------------+-----+
|            Tools|  842|
|    Entertainment|  623|
|        Education|  549|
|          Medical|  463|
|         Business|  460|
|     Productivity|  424|
|           Sports|  398|
|  Personalization|  392|
|    Communication|  387|
|        Lifestyle|  381|
|          Finance|  366|
|           Action|  365|
| Health & Fitness|  340|
|      Photography|  335|
|           Social|  295|
| News & Magazines|  283|
|         Shopping|  260|
|   Travel & Local|  257|
|           Dating|  234|
|Books & Reference|  231|
+-----------------+-----+
only showing top 20 rows



In [9]:
df.count()

10841

In [10]:
# Outliers - I cannot chack outliers until I don't transform the data to numbers

df.select("Rating").summary("min", "25%", "50%", "75%", "max").show()

+-------+-----------+
|summary|     Rating|
+-------+-----------+
|    min| navigation|
|    25%|        4.1|
|    50%|        4.4|
|    75%|        4.6|
|    max|        NaN|
+-------+-----------+



#### Data Quality Assessment

In [11]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [12]:
# Null values per column
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  0|       0|     0|      0|   0|       0|   0|    0|             1|     0|           0|          1|          1|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+



In [13]:
# Duplicate rows
df.groupBy(df.columns).count().filter("count > 1").show()

+--------------------+------------------+------+-------+------------------+------------+----+-----+--------------+--------------------+----------------+------------------+------------------+-----+
|                 App|          Category|Rating|Reviews|              Size|    Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver|       Android Ver|count|
+--------------------+------------------+------+-------+------------------+------------+----+-----+--------------+--------------------+----------------+------------------+------------------+-----+
|Viki: Asian TV Dr...|     ENTERTAINMENT|   4.3| 407719|Varies with device| 10,000,000+|Free|    0|          Teen|       Entertainment|   July 27, 2018|Varies with device|Varies with device|    2|
|          Bowmasters|              GAME|   4.7|1535973|Varies with device| 50,000,000+|Free|    0|          Teen|              Action|   July 23, 2018|            2.12.5|        4.1 and up|    2|
|   School of D

In [14]:
# It is very difficult to see above but there are duplicates

df.groupBy(df.columns).count().filter("count > 1").select("count").show()

+-----+
|count|
+-----+
|    2|
|    2|
|    2|
|    4|
|    2|
|    2|
|    2|
|    4|
|    2|
|    3|
|    2|
|    2|
|    2|
|    2|
|    3|
|    2|
|    2|
|    2|
|    2|
|    2|
+-----+
only showing top 20 rows



In [15]:
# Applying some small cleaning, I notice there is some garbage at the end but I don't pay attention 
# because I will LIMIT 10 or so to work with the data

from pyspark.sql.functions import trim, lower

df.groupBy(trim(lower("Category")).alias("Category_clean")) \
  .count() \
  .orderBy("count", ascending=False) \
  .show(100, truncate=False)


+-------------------+-----+
|Category_clean     |count|
+-------------------+-----+
|family             |1972 |
|game               |1144 |
|tools              |843  |
|medical            |463  |
|business           |460  |
|productivity       |424  |
|personalization    |392  |
|communication      |387  |
|sports             |384  |
|lifestyle          |382  |
|finance            |366  |
|health_and_fitness |340  |
|photography        |335  |
|social             |295  |
|news_and_magazines |283  |
|shopping           |260  |
|travel_and_local   |258  |
|dating             |234  |
|books_and_reference|231  |
|video_players      |175  |
|education          |156  |
|entertainment      |149  |
|maps_and_navigation|136  |
|food_and_drink     |127  |
|house_and_home     |88   |
|libraries_and_demo |85   |
|auto_and_vehicles  |85   |
|weather            |82   |
|art_and_design     |65   |
|events             |64   |
|comics             |60   |
|parenting          |60   |
|beauty             

#### Problem Identification

List of anomalies, prioritization and assumptions

1) Eliminate Duplicates before cleaning
2) After running describe, summary and confirmed by schema, there are a lot of string must be transform to integer
3) After running select, there are few nulls to take care / very few from 10841 rows
4) Is there a relationship between app size and rating? Those two variables are string, thay need to be converted to int
5) Rating has non-numeric inputs to eliminate
6) Eliminate Price = 0 (non-sense)
7) Can we identify underperforming categories (low installs, low ratings)? Those two variables are string, thay need to be converted to int
8) I notice ENTERTAINMENT is a category in the database but it is not in the list, so I need to apply trim and lower
9) Applying some small cleaning, I notice there is some garbage at the end but I don't pay attention, I will LIMIT 10 or so to work with the data

<div style="background-color:#fff5e6; padding:15px; border-radius:8px">
List of prompts used in data discovery

1) Give me an example of function to identify outliers in pyspark
2) Give me a way to identify Nulls, how many and where they are in my df with pysparks
3) Installs:	Remove commas and + signs. Example: "1,000+" → 1000.
4) Size	Convert "14k" → 0.014, "19M" → 19, "Varies with device" → null
5) Price	Remove $ symbol.
6) Rating	Remove or filter out non-numeric entries.
7) I don't have product_id or something similar, how can I detect duplicates?
8) from df.groupBy(*df.columns).count().filter("count > 1").show(), show me the column 'count' / it was difficult to see
9) df.groupBy("Category").count().orderBy("count", ascending=False).show() does not bring ENTERTAINMENT
10) which column I need to transform to float or integer? root

## Data Preparation & Validation

| Column     | Transformation Suggestion                               |
| ---------- | ------------------------------------------------------- |
| `Size`     | Convert `"k"`/`"M"` → float MB                          |
| `Rating`   | Cast to `float` after filtering bad values              |
| `Installs` | `regexp_replace` commas and `+`, cast to `int`          |
| `Price`    | (optional for advanced use) remove `$`, cast to `float` |
| `Category` | Trim & lowercase                                        |
                        

*Source ChatGPT after my prompts

In [16]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [17]:
# 1) Eliminate Duplicates
df.count() # before eliminating

10841

In [18]:
df_cleaned = df.dropDuplicates()
df_cleaned.count()

10358

In [19]:
# 2) Eliminate Nulls in columns
df_cleaned = df.na.drop()
df_cleaned.count()

10839

In [20]:
# 3 Cleaning Size_MB
df_cleaned = df_cleaned.withColumn("Size", 
                                   when(col("Size").contains("M"), regexp_replace("Size", "M", "").cast("float"))
                                   .when(col("Size").contains("k"), regexp_replace("Size", "k", "").cast("float") / 1024)
                                   .otherwise(None))
df_cleaned.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: double (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [21]:
# 4 Clean and convert Installs
df_cleaned = df_cleaned.withColumn("Installs", regexp_replace("Installs", "[+,]", "").cast("int"))
df_cleaned.count()

10839

In [22]:
# After trasnforming, let's drop those that are still a string
df_cleaned = df_cleaned.filter(col("Rating").rlike("^\d+(\.\d+)?$")) \
                       .withColumn("Rating_clean", col("Rating").cast("float"))
df_cleaned.count()

9363

In [23]:
# 5. Clean and convert Rating / Eliminate Nulls also
df_cleaned = df_cleaned.withColumn("Rating", col("Rating").cast("float")).filter(col("Rating").isNotNull())       
df_cleaned.count()

9363

In [24]:
# 6. Clean and convert Price / let's also drop those prices = 0
# I decide to leave price cero and deal with that in Tableau because there are too many +8k 
#It looks most of them are Free games from the store

df_cleaned = df_cleaned.withColumn("Price", regexp_replace("Price", "\\$", "").cast("float"))
df_cleaned.count()

9363

In [25]:
# 7. Normalize Category (AND Rename)
df_cleaned = df_cleaned.withColumn("Category", lower(trim(col("Category")))).withColumnRenamed("Category", "Category_clean")
df_cleaned.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category_clean: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: double (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)
 |-- Rating_clean: float (nullable = true)



#### SQL to create a file to Tableau

In [26]:
df_cleaned.createOrReplaceTempView("sales_table")

In [28]:
df_clean_SQL = spark.sql("""
    SELECT App, Category_clean, Rating, Size, Installs, Price, Genres
    FROM sales_table        
    ORDER BY Size DESC     
""")

df_clean_SQL.show(5)

+--------------------+--------------+------+-----+---------+-----+--------------------+
|                 App|Category_clean|Rating| Size| Installs|Price|              Genres|
+--------------------+--------------+------+-----+---------+-----+--------------------+
|Hungry Shark Evol...|          game|   4.5|100.0|100000000|  0.0|              Arcade|
|The Walking Dead:...|          game|   4.0|100.0|  1000000|  0.0|              Action|
|Navi Radiography Pro|       medical|   4.7|100.0|      500|15.99|             Medical|
|Talking Babsy Bab...|     lifestyle|   4.0|100.0| 10000000|  0.0|Lifestyle;Pretend...|
|Hungry Shark Evol...|          game|   4.5|100.0|100000000|  0.0|              Arcade|
+--------------------+--------------+------+-----+---------+-----+--------------------+
only showing top 5 rows



In [29]:
# Save cleaned data file for Tableau usage
df_clean_SQL.write.option("header", True).mode("overwrite").csv("output/clean_apps_data")

In [30]:
import os
os.getcwd()

'/home/jovyan'

<div style="background-color:#fff5e6; padding:15px; border-radius:8px">
List of prompts used in Data Preparation & Validation

1) According to my business questions, what are the columns I need to check thay are clean.
2) from pyspark.sql.functions import ( ... include here eliminate the old column
3) df_cleaned = df_cleaned.withColumn("Rating", col("Rating").cast("float")).filter(col("Rating").isNotNull())
   but it did not work because after converting, I need to drop string
4) df_cleaned = df_cleaned.withColumn("Price", regexp_replace("Price", "\$", "").cast("float")) ... I want to eliminate price = cero
5) I want to save this in a CSV file... using pyspark

<div style="background-color:#fff5e6; padding:15px; border-radius:8px">

Summary of the prompts and actions to copy the CSV file from the container to my folder in windows

To export my PySpark DataFrame from JupyterLab running in Docker and access it on my Windows machine, I first wrote the DataFrame to disk using:

python
Copy
Edit
df_clean_SQL.coalesce(1).write \
    .option("header", True) \
    .mode("overwrite") \
    .csv("/home/jovyan/output/clean_apps_data")
This saved the output as a folder inside the container at /home/jovyan/output/clean_apps_data.

After verifying the file existed in that path, I exited the container shell and, from my Windows terminal, ran:
bash
Copy
Edit
docker cp peaceful_galois:/home/jovyan/output/clean_apps_data C:\Users\aleja\pyspark-docker

That copied the entire folder to my desired location on Windows. When I only wanted the CSV file itself, I used:
bash
Copy
Edit
docker cp peaceful_galois:/home/jovyan/output/clean_apps_data/part-00000-*.csv C:\Users\aleja\pyspark-docker\clean_apps_data.csv
This gave me direct access to the PySpark output in a single .csv file I could open and use locally.

## Visual Analysis Development

<div style="background-color:#fff5e6; padding:15px; border-radius:8px">

Now that I’ve exported the cleaned dataset containing the relevant columns (App, Category_clean, Rating, Size, Installs, Price, and Genres), my next step is to import this CSV into Tableau to build the visual analysis.

In Tableau, I’ll focus on developing a minimum of six core visualizations, each addressing one of the following business questions:

Is there a relationship between app size and rating?
→ I’ll create a scatterplot of Size vs Rating, possibly colored by Category_clean or sized by Installs.

Can we identify underperforming categories (low installs, low ratings)?
→ I’ll use boxplots or heatmaps comparing Rating and Installs across Category_clean.

What genres tend to perform better in terms of user engagement (ratings)?
→ I’ll build bar charts or distribution plots to compare average Rating by Genres.

As I design each visualization, I’ll document the choices I make—such as filtering, color use, aggregation logic, and layout—ensuring each chart supports one of the business questions clearly. This will lead to a final interactive dashboard that presents data-driven insights in a clear and accessible format.

### Link to Published Dashboard

https://public.tableau.com/views/Week5-M8-LAB/Welcome?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

## Conclusion

<div style="background-color:#fff5e6; padding:15px; border-radius:8px">

1. Is there a relationship between app size and rating?
No strong relationship was identified between app size and user rating. Both small and large apps tend to center around a 4.2 average rating, suggesting that size does not meaningfully influence perceived quality.
2. Can we identify underperforming categories (low installs, low ratings)?
Yes. The data highlights that certain app categories underperform:
•	Low ratings: Notably seen in Dating, Maps & Navigation, and Video Players.
•	Low installs: Common among Events, Beauty, and Parenting apps.
These categories may benefit from improved user experience or marketing efforts.
3. What genres tend to perform better in terms of user engagement (ratings)?
The top-performing genres in terms of average rating are:
•	Education, followed by Action and Sports.
These genres consistently score higher on user satisfaction and egagement.
